### LLM Model Demo

In [ ]:
# Installed LangChain 
#%pip install langchain langchain-community faiss-cpu langchain-chroma langchain-openai langchain_text_splitters langgraph beautifulsoup4 langchain-huggingface

In [ ]:
#%pip install langchain_huggingface
#%pip install langchain

In [ ]:
# First install any missing packages
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_ollama.llms import OllamaLLM
from langchain_ollama import OllamaEmbeddings


In [2]:
parser = StrOutputParser()
llm = OllamaLLM(
    model="llama3.2",
    top_k=3
    )

In [3]:
messages = [
    SystemMessage(content="Translate the following from English into Bengali"),
    HumanMessage(content="hi!"),
]

In [4]:
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [5]:
result = llm.invoke(messages)
parser.invoke(result)

'সর্বোত্তম শুভেচ্ছা! (Shubho shubho! - শুভেচ্ছা, a common Bengali greeting)\n\n(Note: The word "hi" is not typically used in formal greetings in Bengali. Instead, the phrase "shubho shubho" or simply "shubhobroto" are more commonly used.)'

In [6]:
chain = prompt_template | llm | parser
chain.invoke({"language": "bengali", "text": "how are you doing"})

"আপনি কেমন আছেন? (Aponi kemon achhen?)\n\nHere's a more informal version:\n\nকেমন আছেন? (Kemn achhen?)"

In [7]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

llm = OllamaLLM(
    model="llama3.2",
    top_k=3
    )

chain = prompt | llm

chain.invoke({"question": "How far Earth from Sun?"})

"To determine the average distance of Earth from the Sun, let's break it down into steps:\n\n1. **Understanding Units**: We'll be working with astronomical units (AU). 1 AU is equal to about 93 million miles or 149.6 million kilometers.\n\n2. **Identifying Average Distance**: The average distance between Earth and the Sun is approximately 93 million miles or 149.6 million kilometers. This distance can vary slightly over the course of a year due to the elliptical shape of Earth's orbit, but for most practical purposes, this value serves as our standard.\n\n3. **Additional Context (Optional)**: It's worth noting that the closest point in an Earth-Sun orbit is called perihelion and occurs around early January. At this point, Earth is about 91.5 million miles from the Sun, which is a slightly shorter distance than its average distance. The farthest point, aphelion, happens in early July when Earth is approximately 94.5 million miles away.\n\nSo, to summarize, the average distance of Earth 

In [ ]:

# Load documents from a local folder
def load_documents_from_folder(folder_path):
    #loader = DirectoryLoader(folder_path,glob="**/*.pdf",show_progress=True)
    loader = PyPDFDirectoryLoader(folder_path)
    documents = loader.load()
    return documents

# Embed and vectorize documents using FAISS
def embed_and_vectorize_documents(documents):
    embeddings = OllamaEmbeddings(model="llama3.2")
    vector_store = FAISS.from_documents(documents, embeddings)
    return vector_store

# Implement RAG
def implement_rag(vector_store, query):
    llm  = OllamaLLM(
    model="llama3.2"
    )
    retriever = vector_store.as_retriever()
    rag_chain = RetrievalQA(llm=llm, retriever=retriever)
    result = rag_chain.run(query)
    return result

messages = [
    SystemMessage(content="As Technical Recruiter Evaluate the following resumes"),
    HumanMessage(content="Hello Recruiter"),
]

system_template = "Translate the following into {language}:"

"""prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
) """

if __name__ == "__main__":
    folder_path = r"C:\Users\HSASS\OneDrive - Wipro\Desktop\Temp2\Resume"
    query = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")])

    # Load documents
    documents = load_documents_from_folder(folder_path)

    # Embed and vectorize documents
    vector_store = embed_and_vectorize_documents(documents)

    # Implement RAG and get the result
    result = implement_rag(vector_store, query)
    print(result)
    